In [1]:
import sys
from ortools.sat.python import cp_model
from numpy import *

In [2]:
board = [
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],
    
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,1, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 2,0,0],
    
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],
    [0,0,0, 0,0,0, 0,0,0],
]
def solve_miracle_sudoku(board):
    '''
    Solves an n by n sudoku board using standard sudoku rules and prints a board if there is a feasible solution
    Rule 1: Sudoku cells can only contain numbers from 1 to n
    Rule 2: No value can repeat in a given row or column
    Rule 3: No value can repeat in a given sqrt(n) x sqrt(n) quadrant
    Rule 4a: Any two cells separated by a knight's move (in chess) cannot contain the same digit.
    Rule 4b: Any two cells separated by a king's move (in chess) cannot contain the same digit.
    Rule 5: Any two orthogonally adjacent cells cannot contain consecutive digits.
    '''

    board_size = len(board)
    
    model = cp_model.CpModel()
    # Creates the variables.
    # The array index is the column, and the value is the row.
    # Rule 1: Sudoku cells can only contain numbers from 1 to n
    cboard = array(
        [[model.NewIntVar(1, board_size, f'x{row},{col}') for col in range(board_size)] for row in range(board_size)]
    )
    
    # Create a constraint for each known value in the board
    for i in range(board_size):
        for j in range(board_size):
            if board[i][j] != 0:
                model.Add(cboard[i,j] == board[i][j])
    
    # Rule 2: No value can repeat in a given row or column
    for i in range(board_size):
        model.AddAllDifferent(cboard[i,:])
        model.AddAllDifferent(cboard[:,i])
        
    # Rule 3: No value can repeat in a given sqrt(n) x sqrt(n) quadrant
    sq_len = int(sqrt(board_size))
    for i in range(sq_len):
        for j in range(sq_len):
            row_st, row_ed = sq_len*i, sq_len*i+sq_len
            col_st, col_ed = sq_len*j, sq_len*j+sq_len
            model.AddAllDifferent(cboard[row_st:row_ed, col_st:col_ed].flatten())

    # Rule 4a: Any two cells separated by a knight's move (in chess) cannot contain the same digit.
    for i in range(board_size):
        for j in range(board_size):
            if i >= 2:
                if j >=1:
                    model.Add(cboard[i,j] != cboard[i-2,j-1])
                if j <= board_size - 2:
                    model.Add(cboard[i,j] != cboard[i-2,j+1])
            if i <= board_size - 3:
                if j >=1:
                    model.Add(cboard[i,j] != cboard[i+2,j-1])
                if j <= board_size - 2:
                    model.Add(cboard[i,j] != cboard[i+2,j+1])
            if j >= 2:
                if i >=1:
                    model.Add(cboard[i,j] != cboard[i-1,j-2])
                if i <= board_size - 2:
                    model.Add(cboard[i,j] != cboard[i+1,j-2])
            if j <= board_size - 3:
                if i >=1:
                    model.Add(cboard[i,j] != cboard[i-1,j+2])
                if i <= board_size - 2:
                    model.Add(cboard[i,j] != cboard[i+1,j+2])
    # Rule 4b: Any two cells separated by a king's move (in chess) cannot contain the same digit.
            for i2 in range(max(0,i-1), min(board_size,i+2)):
                for j2 in range(max(0,j-1), min(board_size,j+2)):
                    if i != i2 and j != j2:
                        model.Add(cboard[i,j] != cboard[i2,j2])
    # Rule 5: Any two orthogonally adjacent cells cannot contain consecutive digits.
            if i-1 >= 0:
                model.Add(cboard[i,j] + 1 != cboard[i-1,j])
                model.Add(cboard[i,j] - 1 != cboard[i-1,j])
            if i+1 <= board_size - 1:
                model.Add(cboard[i,j] + 1 != cboard[i+1,j])
                model.Add(cboard[i,j] - 1 != cboard[i+1,j])
            if j-1 >= 0:
                model.Add(cboard[i,j] + 1 != cboard[i,j-1])
                model.Add(cboard[i,j] - 1 != cboard[i,j-1])
            if j+1 <= board_size - 1:
                model.Add(cboard[i,j] + 1 != cboard[i,j+1])
                model.Add(cboard[i,j] - 1 != cboard[i,j+1])
            
    ### Solve model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    ## Print solution
    if status != cp_model.INFEASIBLE:
        for i in range(board_size):
            for j in range(board_size):
                end = " " if j % 3 == 2 else ""
                print('%i,' % solver.Value(cboard[i,j]), end=end)
            print()
            if i % 3 == 2:
                print()
        print()
    else:
        print(status)


solve_miracle_sudoku(board)

4,8,3, 7,2,6, 1,5,9, 
7,2,6, 1,5,9, 4,8,3, 
1,5,9, 4,8,3, 7,2,6, 

8,3,7, 2,6,1, 5,9,4, 
2,6,1, 5,9,4, 8,3,7, 
5,9,4, 8,3,7, 2,6,1, 

3,7,2, 6,1,5, 9,4,8, 
6,1,5, 9,4,8, 3,7,2, 
9,4,8, 3,7,2, 6,1,5, 


